In [15]:
!pip install geopandas requests numpy folium geopy osmnx h3
!pip install dbfread


In [1]:


import geopandas as gpd
import pandas as pd
import json
import requests
import numpy as np
from geopy.geocoders import Nominatim, Here
from geopy.exc import GeocoderTimedOut
from ipykernel import kernelapp as app
from geopy import distance
import osmnx as ox
from shapely import wkt
import matplotlib
from folium.plugins import MarkerCluster
import random
import json
from shapely.geometry import Polygon
from h3 import h3
import folium
import warnings

from dbfread import DBF



In [118]:
### floats from polygon str

import re

s = 'SRID=4326;POLYGON((131.95343596 44.75546995,131.95174026 44.7675457,131.9757843 44.7694273,131.97756764 44.75734203,131.95343596 44.75546995))'


def get_floats(s):
    return(list(map(float,re.split(' |,', s[19:-2]))))
    
def get_floats_x(s):
    floats = list(map(float,re.split(' |,', s[19:-2])))
    ret = []
    for i,f in enumerate(floats):
        if i%2 == 0:
            ret.append(f)
    return ret
    
def get_floats_y(s):
    floats = list(map(float,re.split(' |,', s[19:-2])))
    ret = []
    for i,f in enumerate(floats):
        if i%2 == 1:
            ret.append(f)
    return ret
    
def get_centroid(s):
    floats = list(map(float,re.split(' |,', s[19:-2])))
    
    x=0
    y=0
    
    for i,f in enumerate(floats):
        if i%2 == 0:
            x+=f
        else:
            y+=f
    
    x = 2 * x / len(floats)
    y = 2 * y / len(floats)
    
    return x,y

print(get_floats(s))
print(get_floats_x(s))
print(get_floats_y(s))
print(get_centroid(s))


[131.95343596, 44.75546995, 131.95174026, 44.7675457, 131.9757843, 44.7694273, 131.97756764, 44.75734203, 131.95343596, 44.75546995]
[131.95343596, 131.95174026, 131.9757843, 131.97756764, 131.95343596]
[44.75546995, 44.7675457, 44.7694273, 44.75734203, 44.75546995]
(131.962392824, 44.761050985999994)


In [2]:
data = DBF("fires.dbf")
df = pd.DataFrame(data)

In [100]:
df['active_fir'] = df['active_fir'].apply(lambda x: x.replace('POLYGON ((', 'POLYGON(('))

In [128]:
t = df.groupby(['year','fireid'])

len(t)


for i,u in enumerate(t):
    if i > 0:
        break

    print(u[1]['active_fir'].iloc[0])
    print(get_floats(u[1]['active_fir'].iloc[0]))
    print(get_centroid(u[1]['active_fir'].iloc[0]))
    print(get_centroid(u[1]['active_fir'].iloc[1]))
    

SRID=4326;POLYGON((60.0275099 57.93380566,60.02013111 57.94353771,60.03782368 57.95276451,60.05432892 57.9550972,60.06373882 57.95265293,60.07104231 57.94292564,60.06889854 57.93866843,60.03590902 57.93399542,60.0275099 57.93380566))
[60.0275099, 57.93380566, 60.02013111, 57.94353771, 60.03782368, 57.95276451, 60.05432892, 57.9550972, 60.06373882, 57.95265293, 60.07104231, 57.94292564, 60.06889854, 57.93866843, 60.03590902, 57.93399542, 60.0275099, 57.93380566]
(60.04521024444445, 57.94302812888889)
(60.04611865166667, 57.942284335000004)


In [90]:
t = df.groupby(['year','fireid','duration'], as_index = False).agg({'dt':'count'}).rename(columns = {'dt':'count_dates'})


,year,fireid,duration,count_dates
0,2009,816,5,2
1,2009,904,14,8
2,2009,1008,94,40
3,2009,1323,6,2
4,2009,1352,32,19
...,...,...,...,...
38514,2020,49292,6,3
38515,2020,49295,4,4
38516,2020,49369,4,3
38517,2020,49499,4,3


In [137]:
priv = DBF("result_format.dbf")
rf = pd.DataFrame(priv)

In [138]:
rf

,fireid
0,1212
1,1654
2,1753
3,1549
4,1769
...,...
96,1515
97,1208
98,1210
99,1760


In [180]:
priv = DBF("validate_public.dbf")
pr = pd.DataFrame(priv)

In [181]:
pr

,fireid,dt,forecast_d
0,1212,2021-05-05,2021-05-10
1,1654,2021-07-30,2021-08-04
2,1753,2021-07-16,2021-07-21
3,1549,2021-07-27,2021-08-01
4,1769,2021-08-27,2021-09-01
...,...,...,...
96,1963,2021-07-09,2021-07-14
97,1515,2021-07-30,2021-08-04
98,1208,2021-05-04,2021-05-09
99,2113,2021-07-24,2021-07-29


In [174]:
final = pd.read_csv('final_pred.csv')

In [179]:
for u in final['fireid'].values:
    print(u)

1212
1654
1753
1549
1769
2134
1014
1088
1910
1782
1562
1176
1839
676
2195
810
2008
1294
1269
1730
1891
1302
1759
2220
1306
1931
1796
1298
1688
2137
565
947
591
2006
1611
1827
523
519
924
1732
219
2180
1504
1455
2156
1766
1840
1754
1090
877
2136
1618
1225
1570
955
2019
1695
1296
2043
1673
1854
1816
55
508
2022
1060
2190
1012
989
1945
1256
1204
1220
1837
1196
2013
1029
1812
1615
1767
1245
1647
1492
1676
1506
1578
1760
1163
758
2189
299
1210
1593
1723
1811
1809
1963
1515
1208
2113
1560
